In [62]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [63]:
# Download the VADER lexicon for sentiment analysis if not already present.
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MBR\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [64]:
 # Establish the connection to the database
Customer_Reviews =pd.read_csv('Customer Reviews.csv')
Customer_Reviews.head(0)
                            

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText


In [65]:
 # Define the SQL query to fetch customer reviews data
query=Customer_Reviews.head(0)
query

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText


In [66]:
# Initialize the VADER sentiment intensity analyzer for analyzing the sentiment of text data
sia = SentimentIntensityAnalyzer()

# Define a function to calculate sentiment scores using VADER
def calculate_sentiment(review):
    # Get the sentiment scores for the review text
    sentiment = sia.polarity_scores(review)
    # Return the compound score, which is a normalized score between -1 (most negative) and 1 (most positive)
    return sentiment['compound']

In [67]:
# Define a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    # Use both the text sentiment score and the numerical rating to determine sentiment category
    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  # High rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive'  # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'  # Low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment

In [68]:
# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

In [69]:
# Apply sentiment analysis to calculate sentiment scores for each review
Customer_Reviews['SentimentScore'] = Customer_Reviews['ReviewText'].apply(calculate_sentiment)



In [70]:
# Apply sentiment categorization using both text and rating
Customer_Reviews['SentimentCategory'] = Customer_Reviews.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)


In [71]:
# Apply sentiment bucketing to categorize scores into defined ranges
Customer_Reviews['SentimentBucket'] = Customer_Reviews['SentimentScore'].apply(sentiment_bucket)


In [72]:
# Display the first few rows of the DataFrame with sentiment scores, categories, and buckets
print(Customer_Reviews.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                             ReviewText  SentimentScore SentimentCategory  \
0  Average experience, nothing special.         -0.3089    Mixed Negative   
1            The quality is  top-notch.          0.0000          Positive   
2    Five stars for the quick delivery.          0.0000          Positive   
3   Good quality, but could be cheaper.          0.2382    Mixed Positive   
4  Average experience, nothing special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [73]:
Customer_Reviews.head(0)

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket


In [74]:
# Save the DataFrame with sentiment scores, categories, and buckets to a new CSV file
Customer_Reviews.to_csv('customer_reviews_with_sentiment.csv', index=False)